### 07. AI-agent

- En AI-agent är ett system/program som kan agera autonomt för att uppnå ett mål
    - ta in information (input),
    - fatta beslut baserat om vad som ska göras (LLM står för logiken)
    - utföra åtgärder (använda 'tools').

https://python.langchain.com/docs/integrations/chat/google_generative_ai/

![07 diagram](docs/07.drawio.png)

In [1]:
import os
import json

from dotenv import load_dotenv
import httpx

from langchain_google_genai import ChatGoogleGenerativeAI
from langchain_core.tools import tool
from langchain_core.messages import HumanMessage
from langgraph.checkpoint.memory import MemorySaver
from langgraph.prebuilt import create_react_agent


load_dotenv()
API_KEY = os.getenv("GOOGLE_API_KEY")

In [2]:
# obs: i detta exempel använder vi LangChain och LangGraph som är populära ramverk för bygga agenter (speciellt i python)
# i .net-världen är Semantic Kernel ett populärt alternativ

In [3]:
@tool
def get_location_coordinates(location: str) -> dict:
    """returns coordinates in latitude and longitude for the given location

    Args:
        location (str): the location

    Returns:
        dict: latitude and longitude
    """

    url = f"https://maps.googleapis.com/maps/api/geocode/json?address={location}&key={API_KEY}"

    response = httpx.get(url)
    result = json.loads(response.text)
    location = result["results"][0]["geometry"]["location"]

    return {"latitude": location["lat"], "longitude": location["lng"]}

In [4]:
@tool
def get_current_temperature(latitude: str, longitude: str) -> dict:
    """return the current temperatire for the given coordinates

    Args:
        latitude (str): latitude
        longitude (str): longitude

    Returns:
        dict: the temperature
    """

    url = f"https://weather.googleapis.com/v1/currentConditions:lookup?key={API_KEY}&location.latitude={latitude}&location.longitude={longitude}"
    response = httpx.get(url)
    result = json.loads(response.text)

    return {
        "temperature": result["temperature"]["degrees"],
        "unit": result["temperature"]["unit"],
    }

In [5]:
memory = MemorySaver()
client = ChatGoogleGenerativeAI(model="gemini-2.0-flash", api_key=API_KEY)
tools = [get_location_coordinates, get_current_temperature]

agent_executor = create_react_agent(client, tools, checkpointer=memory)

In [ ]:
config = {"configurable": {"thread_id": "abc123"}}

for step in agent_executor.stream(
    {"messages": [HumanMessage(content="what is the current temperature in moheda?")]},
    config=config,
    stream_mode="values",
):
    step["messages"][-1].pretty_print()

In [ ]:
config = {"configurable": {"thread_id": "abc1234"}}

for step in agent_executor.stream(
    {"messages": [HumanMessage(content="what is the current temperature in moheda and lessebo?")]},
    config=config,
    stream_mode="values",
):
    step["messages"][-1].pretty_print()